In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [65]:
import pandas as pd
import numpy as np
import random
from scipy.stats import mode

In [66]:
train_df = pd.read_csv("../Data/predict_student_performance_from_game_play/train.csv")

In [67]:
test_df = pd.read_csv("../Data/predict_student_performance_from_game_play/test.csv")

In [68]:
train_labels = pd.read_csv("../Data/predict_student_performance_from_game_play/train_labels.csv")

In [69]:
train_labels.shape

(424116, 2)

In [70]:
print(train_df.shape)
print(test_df.shape)

(26296946, 20)
(3728, 21)


In [71]:
train_df.head(20)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
5,20090312431273200,5,3423,person_click,basic,0,NaN,-412.991405,-157.314686,381.0,492.0,NaN,"Sure thing, Jo. Grab your notebook and come up...",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
6,20090312431273200,6,5197,person_click,basic,0,NaN,478.485079,-199.971679,593.0,485.0,NaN,"See you later, Teddy.",teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4
7,20090312431273200,7,6180,person_click,basic,0,NaN,503.355128,-168.619913,609.0,453.0,NaN,I get to go to Gramps's meeting!,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4
8,20090312431273200,8,7014,person_click,basic,0,NaN,510.733442,-157.720642,615.0,442.0,NaN,Now where did I put my notebook?,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4
9,20090312431273200,9,7946,person_click,basic,0,NaN,512.048005,-153.743631,616.0,438.0,NaN,\u00f0\u0178\u02dc\u00b4,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4


In [72]:
test_df.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,session_level
0,20090109393214576,0,0,cutscene_click,basic,0,NaN,-413.991405,75.685314,380.0,...,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4,0
1,20090109393214576,1,1965,person_click,basic,0,NaN,-105.991405,-63.314686,688.0,...,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,0
2,20090109393214576,2,3614,person_click,basic,0,NaN,-418.991405,47.685314,375.0,...,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,0
3,20090109393214576,3,5330,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,0
4,20090109393214576,4,6397,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,0


In [73]:
train_df.dtypes

session_id          int64
index               int64
elapsed_time        int64
event_name         object
name               object
level               int64
page              float64
room_coor_x       float64
room_coor_y       float64
screen_coor_x     float64
screen_coor_y     float64
hover_duration    float64
text               object
fqid               object
room_fqid          object
text_fqid          object
fullscreen          int64
hq                  int64
music               int64
level_group        object
dtype: object

In [74]:
test_df.dtypes

session_id          int64
index               int64
elapsed_time        int64
event_name         object
name               object
level               int64
page              float64
room_coor_x       float64
room_coor_y       float64
screen_coor_x     float64
screen_coor_y     float64
hover_duration    float64
text               object
fqid               object
room_fqid          object
text_fqid          object
fullscreen          int64
hq                  int64
music               int64
level_group        object
session_level       int64
dtype: object

In [75]:
train_df.describe()

,session_id,index,elapsed_time,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,fullscreen,hq,music
count,2.629695e+07,2.629695e+07,2.629695e+07,2.629695e+07,564544.000000,2.422367e+07,2.422367e+07,2.422367e+07,2.422367e+07,2.002244e+06,2.629695e+07,2.629695e+07,2.629695e+07
mean,2.113249e+16,6.445102e+02,4.016694e+06,1.219169e+01,3.166559,-5.490579e+01,-1.162119e+02,4.584098e+02,3.855860e+02,3.471003e+03,1.370037e-01,1.202047e-01,9.293778e-01
std,5.712953e+14,5.457880e+02,3.127032e+07,6.497789e+00,2.063291,5.202926e+02,2.184710e+02,2.479389e+02,1.298166e+02,4.602024e+05,3.438513e-01,3.252008e-01,2.561926e-01
min,2.009031e+16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-1.992355e+03,-9.181623e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.101022e+16,2.880000e+02,4.397000e+05,6.000000e+00,1.000000,-3.528395e+02,-2.127183e+02,2.690000e+02,3.040000e+02,9.900000e+01,0.000000e+00,0.000000e+00,1.000000e+00
50%,2.104021e+16,5.950000e+02,1.014801e+06,1.300000e+01,3.000000,-1.116280e+01,-9.760534e+01,4.470000e+02,3.970000e+02,4.190000e+02,0.000000e+00,0.000000e+00,1.000000e+00
75%,2.110052e+16,8.960000e+02,1.747040e+06,1.800000e+01,5.000000,2.966515e+02,2.268786e+01,6.630000e+02,4.710000e+02,1.266000e+03,0.000000e+00,0.000000e+00,1.000000e+00
max,2.210022e+16,2.047300e+04,1.988607e+09,2.200000e+01,6.000000,1.261774e+03,5.436164e+02,1.919000e+03,1.440000e+03,2.451637e+08,1.000000e+00,1.000000e+00,1.000000e+00


In [76]:
test_df.describe()

,session_id,index,elapsed_time,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,fullscreen,hq,music,session_level
count,3.728000e+03,3728.000000,3.728000e+03,3728.000000,153.000000,3366.000000,3366.000000,3366.000000,3366.000000,353.000000,3728.0,3728.0,3728.0,3728.000000
mean,2.009024e+16,670.427039,2.065026e+06,12.171137,3.405229,-14.706281,-131.060856,463.707665,394.447415,1415.631728,0.0,0.0,1.0,4.135730
std,9.599042e+10,411.203747,2.079750e+06,6.282262,2.037277,511.111044,224.031904,243.673841,133.453547,2664.616915,0.0,0.0,0.0,2.423131
min,2.009011e+16,0.000000,0.000000e+00,0.000000,0.000000,-1913.116237,-908.325270,0.000000,10.000000,15.000000,0.0,0.0,1.0,0.000000
25%,2.009011e+16,320.000000,5.708750e+05,7.000000,2.000000,-290.094620,-232.803150,278.250000,320.250000,83.000000,0.0,0.0,1.0,2.000000
50%,2.009031e+16,642.000000,1.174912e+06,11.000000,4.000000,16.843882,-109.000000,460.000000,408.000000,416.000000,0.0,0.0,1.0,4.000000
75%,2.009031e+16,983.250000,2.481634e+06,18.000000,5.000000,355.095522,1.006296,680.000000,486.000000,1316.000000,0.0,0.0,1.0,6.000000
max,2.009031e+16,1563.000000,6.524025e+06,22.000000,6.000000,1214.941972,489.514956,879.000000,658.000000,23451.000000,0.0,0.0,1.0,8.000000


In [77]:
train_df['event_name'].value_counts()

navigate_click        11326433
person_click           6052853
cutscene_click         2703035
object_click           2198211
object_hover           1057085
map_hover               945159
notification_click      649001
notebook_click          564544
map_click               517242
observation_click       212355
checkpoint               71028
Name: event_name, dtype: int64

In [78]:
train_df['name'].value_counts()

undefined    12705785
basic        12648469
close          676696
open           235139
prev            19250
next            11607
Name: name, dtype: int64

In [79]:
train_df['level_group'].value_counts()

13-22    13471703
5-12      8844238
0-4       3981005
Name: level_group, dtype: int64

In [80]:
train_df['fqid'].value_counts()

worker           1878958
archivist        1127341
gramps           1122264
wells             787418
toentry           784496
                  ...   
need_glasses        1297
block_badge_2        993
fox                  465
block_1               37
chap2_finale           1
Name: fqid, Length: 128, dtype: int64

In [81]:
train_df['room_fqid'].value_counts()

tunic.historicalsociety.entry              3622003
tunic.wildlife.center                      3013154
tunic.historicalsociety.cage               2352083
tunic.library.frontdesk                    2145666
tunic.historicalsociety.frontdesk          1943066
tunic.historicalsociety.stacks             1794007
tunic.historicalsociety.closet_dirty       1568994
tunic.humanecology.frontdesk               1326890
tunic.historicalsociety.basement           1168090
tunic.kohlcenter.halloffame                1096872
tunic.library.microfiche                    995674
tunic.drycleaner.frontdesk                  981451
tunic.historicalsociety.collection          957400
tunic.historicalsociety.closet              857273
tunic.flaghouse.entry                       833793
tunic.historicalsociety.collection_flag     452397
tunic.capitol_1.hall                        445569
tunic.capitol_0.hall                        416469
tunic.capitol_2.hall                        326095
Name: room_fqid, dtype: int64

In [82]:
train_df['text_fqid'].value_counts()

tunic.historicalsociety.cage.confrontation               666557
tunic.wildlife.center.crane_ranger.crane                 470755
tunic.historicalsociety.frontdesk.archivist.newspaper    429762
tunic.historicalsociety.entry.groupconvo                 427180
tunic.wildlife.center.wells.nodeer                       399967
                                                          ...  
tunic.capitol_2.hall.chap4_finale_c                         280
tunic.drycleaner.frontdesk.block_0                          221
tunic.historicalsociety.entry.gramps.hub                     39
tunic.humanecology.frontdesk.block_1                         13
tunic.drycleaner.frontdesk.block_1                            3
Name: text_fqid, Length: 126, dtype: int64

In [83]:
train_df['session_id'].value_counts()

20110310463989760    19032
20110515090304556     6000
21110313432157404     5435
21010312074134348     4803
20110219555923044     4743
                     ...  
21090109332150240      639
21000013055772070      637
21030108504475890      635
20110112402567744      634
22000610592348800      591
Name: session_id, Length: 23562, dtype: int64

In [90]:
train_df.loc[train_df['session_id']==20110515090304556 ]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
3409131,20110515090304556,699,0,cutscene_click,basic,0,NaN,-159.991405,-244.314686,634.0,579.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,1,1,0-4
3409132,20110515090304556,700,433,cutscene_click,basic,0,NaN,-159.991405,-244.314686,634.0,579.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,1,1,0-4
3409133,20110515090304556,701,850,cutscene_click,basic,0,NaN,-159.991405,-244.314686,634.0,579.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,1,1,0-4
3409134,20110515090304556,702,4316,cutscene_click,basic,0,NaN,-409.991405,-49.314686,384.0,384.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,1,1,0-4
3409135,20110515090304556,703,10766,person_click,basic,0,NaN,-187.991405,139.685314,606.0,195.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,1,1,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415126,20110515090304556,4896,5529119,navigate_click,undefined,22,NaN,305.226661,30.624826,484.0,284.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,1,1,13-22
3415127,20110515090304556,4897,5529387,navigate_click,undefined,22,NaN,275.422340,32.698719,484.0,284.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,1,1,13-22
3415128,20110515090304556,4898,5529983,navigate_click,undefined,22,NaN,249.413656,33.709781,484.0,284.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,1,1,13-22
3415129,20110515090304556,4899,5529803,navigate_click,undefined,22,NaN,218.398237,34.458993,484.0,284.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,1,1,13-22


train_df['room_coor_x']

In [91]:
train_df['index'].value_counts()

79       23549
77       23549
76       23549
80       23549
81       23549
         ...  
10791        1
10792        1
10793        1
10794        1
20473        1
Name: index, Length: 20348, dtype: int64

In [92]:
train_df['room_coor_x'].value_counts()

425.725528    3915
426.725528    3883
427.725528    3775
424.725528    3640
423.725528    3610
              ... 
352.798521       1
281.824240       1
170.086381       1
60.067543        1
252.299653       1
Name: room_coor_x, Length: 17438693, dtype: int64

In [ ]:
train_df.isnull().sum()

session_id               0
index                    0
elapsed_time             0
event_name               0
name                     0
level                    0
page              25732402
room_coor_x        2073272
room_coor_y        2073272
screen_coor_x      2073272
screen_coor_y      2073272
hover_duration    24294702
text              16679807
fqid               8274415
room_fqid                0
text_fqid         16679702
fullscreen               0
hq                       0
music                    0
level_group              0
dtype: int64

In [2]:
test_df.isnull().sum()

NameError: name 'test_df' is not defined

In [3]:
train_df['session_id'].unique()

NameError: name 'train_df' is not defined